# Vector Search - Query

> Use index create by built-in index feature

Azure AI Search:
- Vector Search
- Hybrid Search
- Semantic Search

https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/readme.md

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

> update value of `index_name` to your index

In [2]:
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZSCH_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

index_name = os.environ["AZSCH_INDEX_NAME"]

#azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
#azure_openai_key = os.environ["AZURE_OPENAI_KEY"] if len(os.environ["AZURE_OPENAI_KEY"]) > 0 else None
#azure_openai_embedding_deployment = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"]

## Search query

In [3]:
query = "What is OpenAI assistant api?"  
topK = 3

In [4]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

## Text Search

- vector_queries = None
- search_text = query

In [5]:
search_client = SearchClient(endpoint, index_name, credential=credential)
  
# Text Search
results = search_client.search(  
    search_text=query,  
    vector_queries=None,
    select=["parent_id", "chunk_id", "chunk", "title"],
    top=topK
)  
  
for result in results:  
    print(f"Score: {result['@search.score']}, Title: {result['title']}, chunk_id: {result['chunk_id'][-1]}") 
    print(f"   parent_id: {result['parent_id']}")  
    #print(f"Content: {result['chunk']}")

Score: 11.026696, Title: What is Azure OpenAI Service?, chunk_id: 0
   parent_id: overview.md
Score: 9.364173, Title: What is Azure OpenAI Service?, chunk_id: 1
   parent_id: overview.md
Score: 9.149903, Title: Azure OpenAI Service Assistant API concepts, chunk_id: 0
   parent_id: assistants.md


## Vector Search

> Add __vector profile__ manually in the index

- vector_queries = [vector_query]
- search_text = None

In [6]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

# Pure Vector Search
search_client = SearchClient(endpoint, index_name, credential=credential)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=topK, fields="vector", exhaustive=True)
# Use the below query to pass in the raw vector query instead of the query vectorization
#vector_query = RawVectorQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "chunk", "title"],
    top=topK
)  
  
for result in results:  
    print(f"Score: {result['@search.score']}, Title: {result['title']}, chunk_id: {result['chunk_id'][-1]}") 
    print(f"   parent_id: {result['parent_id']}")  
    #print(f"Content: {result['chunk']}")

Score: 0.879656, Title: Azure OpenAI Service Assistant API concepts, chunk_id: 0
   parent_id: assistants.md
Score: 0.86666685, Title: How to create Assistants with Azure OpenAI Service, chunk_id: 0
   parent_id: assistant.md
Score: 0.86589706, Title: Quickstart - Getting started with Azure OpenAI Assistants (Preview), chunk_id: 0
   parent_id: assistants-quickstart.md


## Hybrid Search

- vector_queries = [vector query]
- search_text = query

In [8]:
# Hybrid Search
#search_client = SearchClient(endpoint, index_name, credential=credential)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=topK, fields="vector", exhaustive=True)
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "chunk", "title"],
    top=topK
)  
  
for result in results:  
    print(f"Score: {result['@search.score']}, Title: {result['title']}, chunk_id: {result['chunk_id'][-1]}") 
    print(f"   parent_id: {result['parent_id']}")  
    #print(f"Content: {result['chunk']}") 

Score: 0.03279569745063782, Title: Azure OpenAI Service Assistant API concepts, chunk_id: 0
   parent_id: assistants.md
Score: 0.024205941706895828, Title: How to create Assistants with Azure OpenAI Service, chunk_id: 0
   parent_id: assistant.md
Score: 0.019187135621905327, Title: Quickstart - Getting started with Azure OpenAI Assistants (Preview), chunk_id: 0
   parent_id: assistants-quickstart.md


## Semantic Search (Hybrid search + semantic reranking)

> Add __semantic configuration__ manually in the index and update the value of `semantic_configuration_name` parameter.

- vector_queries = [vector_query]
- search_text = query
- query_type=QueryType.SEMANTIC,
- semantic_configuration_name='your-semantic-config'

In [11]:
from azure.search.documents.models import (
    QueryType,
    QueryCaptionType,
    QueryAnswerType
)

# Semantic search
#search_client = SearchClient(endpoint, index_name, credential)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=topK, fields="vector", exhaustive=True)

results = search_client.search(  
    search_text=query,
    vector_queries=[vector_query],
    select=["parent_id", "chunk_id", "chunk", "title"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name='semantic-config',
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=topK
)

for result in results:
    print(f"Score: {result['@search.score']}, Title: {result['title']}, chunk_id: {result['chunk_id'][-1]}") 
    print(f"   Reranker Score: {result['@search.reranker_score']}")
    print(f"   parent_id: {result['parent_id']}")  
    #print(f"Content: {result['chunk']}")  

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"   >Caption: {caption.highlights}\n")
        else:
            print(f"   >Caption: {caption.text}\n")

print("Semantic Answers:")
semantic_answers = results.get_answers()
if semantic_answers:
    for answer in semantic_answers:
        if answer.highlights:
            print(f"   >Semantic Answer: {answer.highlights}")
        else:
            print(f"   >Semantic Answer: {answer.text}")
        print(f"   >Semantic Answer Score: {answer.score}\n")

Score: 0.03279569745063782, Title: Azure OpenAI Service Assistant API concepts, chunk_id: 0
   Reranker Score: 3.357616901397705
   parent_id: assistants.md
   >Caption: # Azure OpenAI Assistants API (Preview)  Assistants, a new feature of Azure OpenAI Service, is now available in public preview. While the chat completions API is lightwei... The Assistants<em> API, as </em>the<em> stateful evolution of the chat completion API, provides a solution for these challenges.</em> Assistants API supports persistent automatically managed.

Score: 0.024205941706895828, Title: How to create Assistants with Azure OpenAI Service, chunk_id: 0
   Reranker Score: 3.00740122795105
   parent_id: assistant.md
   >Caption: # Getting started with Azure OpenAI Assistants (Preview)  Azure OpenAI Assistants (Preview) allows you to create<em> AI </em>assistants tailored to your needs through<em> custom instructions and augmented by advanced tools like code interpreter, and custom functions.</em> In this articl